In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

- Utils

In [ ]:
def show_matrix(matrix:np.ndarray):
    print("---------------------------")
    return print(matrix)

- Dataset

In [3]:
K_data = np.array([
    [2 ,2 ,0 ],
    [2 ,2 ,2 ],
    [2 ,0, 0 ],
    [0 ,2, 2 ],
    [0 ,0 ,2 ]
])
show_matrix(K_data)

---------------------------
[[2 2 0]
 [2 2 2]
 [2 0 0]
 [0 2 2]
 [0 0 2]]


- Fréquence relative et marginale

In [4]:
# somme des individus
total_individus = K_data.sum()
# matrice fréquence relative
F_relative = K_data/total_individus
# matrice fréquence marginal ligne et colonne
F_ligne_marginal = F_relative.sum(axis=1).reshape(-1,1)
F_colonne_marginal = F_relative.sum(axis=0).reshape(-1,1)
# matrice total
step_1 = np.hstack((F_relative,F_ligne_marginal))
step_2 = np.vstack((F_colonne_marginal,np.array([1])))

total_matrix = np.vstack((step_1,step_2.T))
show_matrix(total_matrix)

---------------------------
[[0.11111111 0.11111111 0.         0.22222222]
 [0.11111111 0.11111111 0.11111111 0.33333333]
 [0.11111111 0.         0.         0.11111111]
 [0.         0.11111111 0.11111111 0.22222222]
 [0.         0.         0.11111111 0.11111111]
 [0.33333333 0.33333333 0.33333333 1.        ]]


- Nuage profil ligne et colonne simple

In [10]:
profil_ligne = F_relative/F_ligne_marginal
profil_colonne = F_relative/F_colonne_marginal.T

show_matrix(profil_ligne)
show_matrix(profil_colonne)

---------------------------
[[0.5        0.5        0.        ]
 [0.33333333 0.33333333 0.33333333]
 [1.         0.         0.        ]
 [0.         0.5        0.5       ]
 [0.         0.         1.        ]]
---------------------------
[[0.33333333 0.33333333 0.        ]
 [0.33333333 0.33333333 0.33333333]
 [0.33333333 0.         0.        ]
 [0.         0.33333333 0.33333333]
 [0.         0.         0.33333333]]


- Nuage profil ligne et colonne transformé

In [6]:
profil_ligne_transforme = profil_ligne*(1/np.sqrt(F_colonne_marginal.T))
profil_colonne_transforme = profil_colonne*(1/np.sqrt(F_ligne_marginal))

show_matrix(profil_ligne_transforme)
show_matrix(profil_colonne_transforme)

---------------------------
[[0.8660254  0.8660254  0.        ]
 [0.57735027 0.57735027 0.57735027]
 [1.73205081 0.         0.        ]
 [0.         0.8660254  0.8660254 ]
 [0.         0.         1.73205081]]
---------------------------
[[0.70710678 0.70710678 0.        ]
 [0.57735027 0.57735027 0.57735027]
 [1.         0.         0.        ]
 [0.         0.70710678 0.70710678]
 [0.         0.         1.        ]]


- Nuage profil ligne et colonne transformé centrée

In [31]:
centre_gravite_ligne = np.sqrt(F_colonne_marginal)
centre_gravite_colonne =  np.sqrt(F_ligne_marginal)

# matrice des variances covariance ou matrice d'inertie
profil_ligne_transforme_centre = profil_ligne_transforme-centre_gravite_ligne.T 
# matrice des variances covariance ou matrice d'inertie
profil_colonne_transforme_centre = profil_colonne_transforme-centre_gravite_colonne


# show_matrix(profil_ligne_transforme_centre.std())
show_matrix(centre_gravite_ligne.T)
show_matrix(profil_ligne_transforme_centre)

---------------------------
[[0.57735027 0.57735027 0.57735027]]
---------------------------
[[ 0.28867513  0.28867513 -0.57735027]
 [ 0.          0.          0.        ]
 [ 1.15470054 -0.57735027 -0.57735027]
 [-0.57735027  0.28867513  0.28867513]
 [-0.57735027 -0.57735027  1.15470054]]


- ACP composantes  principales  (facteurs  lignes)  du  nuage  des  profils-lignes

In [34]:
"""
La matrice des variances-covariances, 
n'est que la matrice profil_ligne_transform_centre, 
donc l'inertie est la trace de cette matrice
"""
# matrice poids touours matrice carré 
matrice_poids = np.zeros((5,5))
np.fill_diagonal(matrice_poids,F_ligne_marginal)


# matrice YL = Xl TRANSOFRME - CENTRE gravité a pas confondre avec le profil colonne
matrice_centre_ligne = profil_ligne_transforme_centre.T.dot(matrice_poids).dot(profil_ligne_transforme_centre)
matrice_centre_colone =  profil_colonne_transforme_centre.T.dot(matrice_poids).dot(profil_colonne_transforme_centre)

In [39]:
eig_val,eig_vect = np.linalg.eig(matrice_centre_ligne)
eig_val = np.real(eig_val)
eig_vect = np.real(eig_vect)

show_matrix(eig_val)

index = np.argsort(eig_val)[::-1]
eig_val = eig_val[index]
eig_vect = eig_vect[:,index]

# nb_variable = matrice_centre_ligne.shape[1]
taux_inertie = np.diag(matrice_centre_ligne) # trace de la matrice
# show_matrix(taux_inertie)
# le taux d'inertie cumulé est donc égale à 1 (100%)
cumul_taux_inertie = np.cumsum(taux_inertie)

table_info = np.hstack((np.hstack((eig_val.reshape(-1,1),taux_inertie.reshape(-1,1))),cumul_taux_inertie.reshape(-1,1)))
show_matrix(table_info)

---------------------------
[5.00000000e-01 3.06054180e-17 1.66666667e-01]
---------------------------
[[5.00000000e-01 2.77777778e-01 2.77777778e-01]
 [1.66666667e-01 1.11111111e-01 3.88888889e-01]
 [3.06054180e-17 2.77777778e-01 6.66666667e-01]]
